<a href="https://colab.research.google.com/github/abhishekasharma206/BMS/blob/main/InfluxTestFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install influxdb_client

In [2]:
from google.colab import files

In [3]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import io
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

In [4]:
uploaded = files.upload()

Saving BMS_NN_dataset.csv to BMS_NN_dataset.csv


In [5]:
CSV_COLUMN_NAMES = ['Current', 'Temperature', 'Voltage', 'SoC']

In [6]:
df = pd.read_csv(io.BytesIO(uploaded['BMS_NN_dataset.csv']), names = CSV_COLUMN_NAMES, header = 0)

In [9]:
uploaded1 = files.upload()

Saving soc_predictor.h5 to soc_predictor.h5


In [10]:
model = keras.models.load_model('soc_predictor.h5')

In [11]:
df.head()

,Current,Temperature,Voltage,SoC
0,0.377587,35.0,4.11347,0.922144
1,1.026460,15.0,3.55586,0.391589
2,0.968748,40.0,3.65911,0.489832
3,0.952046,10.0,3.63424,0.466164
4,0.650396,20.0,3.94357,0.760481


In [12]:
soc = df.pop('SoC')

In [13]:
df.head()

,Current,Temperature,Voltage
0,0.377587,35.0,4.11347
1,1.026460,15.0,3.55586
2,0.968748,40.0,3.65911
3,0.952046,10.0,3.63424
4,0.650396,20.0,3.94357


In [14]:
df.loc[:, 'Temperature'] = 25

In [15]:
df.head()

,Current,Temperature,Voltage
0,0.377587,25,4.11347
1,1.026460,25,3.55586
2,0.968748,25,3.65911
3,0.952046,25,3.63424
4,0.650396,25,3.94357


In [16]:
voltage = df.pop('Voltage')

In [17]:
temperature = df.pop('Temperature')

In [18]:
df.head()

,Current
0,0.377587
1,1.026460
2,0.968748
3,0.952046
4,0.650396


In [19]:
current = df.pop('Current')

In [20]:
df.head()

""
0
1
2
3
4


In [21]:
x = len(voltage)
print(x)

87501


In [26]:
token = "teXoimWzBPEly43ay2Gp8F3NosQjw6QZPrd_NF-e8Kl5GaeGd_QwerxsuGoDSO5hCeJZtEFystHJCW2t8Qt1Xg=="
org = "1-PART-17"
bucket = "Battery Management"

client = InfluxDBClient(url="https://eu-central-1-1.aws.cloud2.influxdata.com", token=token)

write_api = client.write_api(write_options=SYNCHRONOUS)

ah = 7.5
batcapideal = 94.5
flag=0
parallel_v = [0.0, 0.0, 0.0]
totv = 0
totamp = 0
l = 1000



In [27]:
def predict_soc():
    for j in range(9):
        if j<=2:
            x=0
        elif j>2 and j<=5:
            x=1
        else:
            x=2
        
        soc[j] = model.predict([[cur_cell[j], temp_cell[j], parallel_v[x]]])
        string = 'state of charge ' + str(j)
        point  = Point("Battery") \
            .tag("Type", "State of charge") \
            .field(string, soc[j]) \
    
        write_api.write(bucket, org, point)

for i in range(l):
    vol = voltage[i]
    for j in range(3):
        parallel_v[j] = vol
        string = 'voltage' + str(j)
        point = Point("Battery") \
            .tag("Type", "Voltage") \
            .field(string, parallel_v[j]) \

        write_api.write(bucket, org, point)
        
    totv = 3 * vol
    point = Point("Battery") \
        .tag("Type", "Voltage") \
        .field("Total Voltage", totv) \

    write_api.write(bucket, org, point)
    
    cur = current[i]
    for j in range(9):
        cur_cell[j] = cur
        string = 'current' + str(j)
        point = Point("Battery") \
            .tag("Type","current") \
            .field(string, cur_cell[j]) \
                
        write_api.write(bucket, org, point)
        
    totamp = 3 * cur
    point = Point("Battery") \
        .tag("Type","current") \
        .field("Total Current", totamp) \
                
    write_api.write(bucket, org, point)
    
    temp = temperature[i]
    for j in range(9):
        temp_cell[j] = temperature[i]
        string = 'temperature' + str(i)
        point = Point("Battery") \
            .tag("Type","temperature") \
            .field(string, temp_cell[i]) \
                
        write_api.write(bucket, org, point)
        predict_soc()
        
    def soh_calculation(totv):
    #Battery capacity calculation
        batcappract= ah * totv
        soh = batcappract /94.5
        point = Point("Battery") \
            .tag("Type", "State of Health") \
            .field("SoH", soh)
        
        write_api.write(bucket, org, point)
    
    charge = random.randint(0, 1)
    if charge == 1:
        Error_code = Point("Battery_error") \
            .field("charging", 1)\
        
        write_api.write(bucket, org, Error_code)
    else:
        Error_code = Point("Battery_error") \
            .field("discharging", 1)\
        
        write_api.write(bucket, org, Error_code)

ApiException: ignored